# Test of whether LanguageModelLoader deliver batch where each row is a continuous set of tokens


Status is that the test works with

-MyLanguageModelLoader 

-fastai's LanguageModelLoader, however, throws an exception while indexing the jagged array


In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.text import * 
from languagemodelloader import *

In [3]:
def getAllBatches(data,epochs=1):
    x=None
    for i in range(epochs):
        for xb,yb in data:
            d= xb.data.numpy()
            if x is None: 
                x = xb.data.numpy().copy()
            else:         
                x = np.concatenate((x, xb.data.numpy().copy()),axis=1)
            continue
    return x            

In [4]:
def generateContinuousNumber(bs,seq_len,iterations,minTokens):
    "The data consist of continuously increasing numbers i jagged array with randomly varying layout"
    jagged = []
    countTokens = 0
    totalTokens = bs*seq_len*iterations
    #print(f"totalTokens:{totalTokens}")
    rand_interval=20
    while countTokens < totalTokens:
        nb = totalTokens-countTokens if   totalTokens-countTokens <rand_interval\
                                     else minTokens+int(np.random.random() * rand_interval)
        jagged.append(np.arange(countTokens+1,countTokens+1+nb))
        countTokens = jagged[-1][-1]
    jagged = np.asarray(jagged)    
    return jagged, countTokens

def printJagged(jagged, countTokens):
    print(f"countTokens:{countTokens}")
    for j in jagged: print(j)

In [5]:
def test_datadirection( bs,seq_len,iterations,minTokens, forward=True, nbInoutGenerations=10, log=False):
    for i in range(nbInoutGenerations):
        jagged,countTokens = generateContinuousNumber(bs,seq_len,iterations,minTokens)
        if log: printJagged(jagged, countTokens)
            
        trainIDS = validIDS = jagged
        db = TextLMDataBunch.from_ids( ".", None, trainIDS, validIDS, bptt=seq_len, p_bptt=p_bptt, bs=bs)

        data = MyLanguageModelLoader(dataset=db.train_dl, bptt=seq_len, p_bptt=p_bptt, bs=bs, shuffle=False, 
                                     backwards=not forward, log=log)
        batches = getAllBatches(data)
        if log: display(pd.DataFrame(data=batches))

        #The diff from one to the next column must be 1 for aligned mini-batches with forward indexing of the data
        #(forward is default for LanguageModelLoader ie.: backwards=False) 
        b_diff = batches[:,1:] - batches[:,0:-1]
        if log: display(pd.DataFrame(data=b_diff))
            
        diff_value = 1 if forward else -1
        assert (b_diff.flatten()==diff_value).all()

# Test continouity of tokens in batches loaded forwards and backwards

In [6]:
#b_ptt     = 0 for MyLanguageModelLoader
p_bptt     =  0  #deactivates changes in the sequence length in MyLanguageModelLoader so that we do not wrap around

#create test data so that we can control whether the LanguageModelLoader returns contigous tokens 
#The genrated data must be size so that the batches will not wrap aoround. 
bs         =  9
seq_len    =  3  #=bptt
iterations =  2
minTokens  =  1 #in a rag array

In [9]:
%time test_datadirection( bs, seq_len, iterations, minTokens, forward=True, nbInoutGenerations=10000)

CPU times: user 5.51 s, sys: 10.9 ms, total: 5.52 s
Wall time: 5.52 s


0

In [11]:
%time test_datadirection( bs, seq_len, iterations, minTokens, forward=False, nbInoutGenerations=10000, log=False)

CPU times: user 6.22 s, sys: 23.6 ms, total: 6.25 s
Wall time: 6.25 s
